# Creación de archivos .csv para poblar la base de datos

In [1]:
import pandas as pd
import numpy as np

import data_const as c
import utils as u
import relations as r


## Entidad Cliente

In [2]:
try:
    client = pd.read_csv(c.filepath_client)
    
except:
    print('Creando entidad Cliente')
    np.random.seed(1)

    n_client = c.cardinality['client']
    n_masc = int(np.random.normal(loc=(n_client)/2, scale=1))
    n_fem = n_client - n_masc

    print(f'Cardinalidad de la relación Cliente: {n_client}')
    print(f'Cantidad de hombres en Cliente: {n_masc}')
    print(f'Cantidad de mujeres en Cliente: {n_fem}')

    data_names_masc = pd.read_csv(c.path_masc_names, encoding='latin-1')
    data_names_fem = pd.read_csv(c.path_fem_names, encoding='latin-1')
    data_surnames = pd.read_csv(c.path_surnames, encoding='latin-1')

    masc_names = list(data_names_masc.loc[:, 'nombre'])
    fem_names = list(data_names_fem.loc[:, 'nombre'])
    surnames = list(data_surnames.loc[:, 'apellido'])

    prob_masc = data_names_masc.loc[:, 'frec']/data_names_masc.loc[:, 'frec'].sum()
    prob_fem = data_names_fem.loc[:, 'frec']/data_names_fem.loc[:, 'frec'].sum()
    prob_ap1 = data_surnames.loc[:, 'frec_pri']/data_surnames.loc[:, 'frec_pri'].sum()
    prob_ap2 = data_surnames.loc[:, 'frec_seg']/data_surnames.loc[:, 'frec_seg'].sum()

    # Nombres
    names_masc = list(np.random.choice(masc_names,
                                   n_masc,
                                   p=prob_masc))
    names_fem = list(np.random.choice(fem_names,
                                  n_fem,
                                  p=prob_fem))
    names = names_masc + names_fem

    surnames_1 = np.random.choice(surnames,
                              n_client,
                              p=prob_ap1)
    surnames_2 = np.random.choice(surnames,
                              n_client,
                              p=prob_ap2)
    # Curps
    curp_masc = list(u.get_curps(names_masc,
                             surnames_1[0: n_masc],
                             surnames_2[0: n_masc],
                             sex='M'))
    curp_fem = list(u.get_curps(names_fem,
                            surnames_1[n_masc:],
                            surnames_2[n_masc:],
                            sex='F'))
    curps = curp_masc + curp_fem

    # Direcciones
    cdmx_data = pd.read_csv(c.filepath_raw + '/all_cdmx_data.csv')
    np.random.seed(1)
    to_select = np.random.choice(np.arange(0, cdmx_data.index[-1]), size=n_client, replace=True)
    cdmx_client = cdmx_data.iloc[to_select]
    cdmx_client.astype({'cp': 'float32'})
    
    # Creamos y rellenamos
    cliente = pd.DataFrame(r.dict_to_df_cliente)

    cliente['curp'] = curps
    cliente['nombre'] = names
    cliente['a_paterno'] = surnames_1
    cliente['a_materno'] = surnames_2

    cliente['calle_num'] = list(cdmx_client['calle_num'])
    cliente['municipio'] = list(cdmx_client['alcaldia'])
    cliente['cp'] = np.array(cdmx_client['cp'], dtype=int)
    cliente['colonia'] = list(cdmx_client['colonia'])
    cliente['estado'] = c.address
    
    cliente['tel_casa'] = u.get_tels(n_client)
    cliente['tel_celular'] = u.get_tels(n_client)
    
    #cliente['email'] = ?
    
    cliente['fotografia'] = u.get_photos(curps)
    
    unities = u.get_unity(curps)
    
    cliente['es_alumno'] = unities[0]
    cliente['es_academico'] = unities[1]
    cliente['es_trabajador'] = unities[2]
    
    # Verificamos que todos los curps sean distintos
    assert len(set(list(cliente['curp']))) == n_client
    
    # Escribimos CSV
    
    #cliente.to_csv(c.filepath_client, index=False)

Creando entidad Cliente
Cardinalidad de la relación Cliente: 10000
Cantidad de hombres en Cliente: 5001
Cantidad de mujeres en Cliente: 4999


In [7]:
cliente

,curp,nombre,a_paterno,a_materno,calle_num,municipio,cp,colonia,estado,tel_casa,tel_celular,email,fotografia,facultad,instituto,unidad,es_alumno,es_academico,es_trabajador
0,HEAJ130814MQJYYK64,JOSE LUIS,HEREDIA,ABAD,Angel R Cavada Mz 57 L 42,Álvaro Obregón,15000,Jalalpa Grande,Cuidad de México,5525239171,5601244301,NaN,proyecto/data/photos/photo_HEAJ130814MQJYYK64.png,NaN,NaN,NaN,0,0,1
1,COMA610715MJIFAB46,ALBERTO,CONDE,MONTELLANO,Pacifico 274 Estac 14,Coyoacán,43500,Rosedal,Cuidad de México,5650348782,5504158208,NaN,proyecto/data/photos/photo_COMA610715MJIFAB46.png,NaN,NaN,NaN,0,0,1
2,GOCJ840224MYGGAA14,JOSE,GOMEZ,CANSECO,"Avd. PACIFICO, 350 Dpto. E-001",Coyoacán,43300,EL ROSEDAL,Cuidad de México,5545617376,5515576169,NaN,proyecto/data/photos/photo_GOCJ840224MYGGAA14.jpg,NaN,NaN,NaN,0,0,1
3,GALJ980709MDXEGO95,JORGE,GARCIA,LA,Cartzo 119 Local a,Cuauhtémoc,64000,Sta Maria la Rivera,Cuidad de México,5556058656,5674956547,NaN,proyecto/data/photos/photo_GALJ980709MDXEGO95.jpg,NaN,NaN,NaN,1,0,0
4,ZUSM061106MNWAUO04,MANUEL,ZUÑIGA,SANTOS,Camelia M-7 L-9,Álvaro Obregón,17400,San Clemente,Cuidad de México,5579578976,5544241397,NaN,proyecto/data/photos/photo_ZUSM061106MNWAUO04.jpg,NaN,NaN,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,GACM351019FSZUDT24,MARGARITA,GARCIA,CONTRERAS,Camino a Sta. Cruz Num. 53,Iztapalapa,97900,Lomas de San Lorenzo,Cuidad de México,5618074080,5555738981,NaN,proyecto/data/photos/photo_GACM351019FSZUDT24.jpg,NaN,NaN,NaN,0,0,1
9996,AGDG480102FPUTYC49,GABRIELA,AGUILERA,DEOLARTE,"Prv. EMPERADOR, MZ-1 LT-38",Magdalena Contreras,10369,TIERRA UNIDA,Cuidad de México,5576540598,5561474681,NaN,proyecto/data/photos/photo_AGDG480102FPUTYC49.jpg,NaN,NaN,NaN,0,0,1
9997,GUCM621030FCLKJT65,MARIA DE LOUR,GUTIERREZ,CARDENAS,Darwin 48,Miguel Hidalgo,11590,Anzures,Cuidad de México,5597321785,5508073714,NaN,proyecto/data/photos/photo_GUCM621030FCLKJT65.png,NaN,NaN,NaN,0,0,1
9998,MOAL150709FCCEBD70,LAURA ESTELA,MOJICA,ALVAREZ,"2DO AND 2 DE ABRIL, MZ-5 LT-3-A",Magdalena Contreras,10900,PUEBLO SAN NICOLAS TOTOLAPAN,Cuidad de México,5549527393,5563656680,NaN,proyecto/data/photos/photo_MOAL150709FCCEBD70.png,NaN,NaN,NaN,0,1,0


In [4]:
cliente['es_alumno'].sum()

1695

In [5]:
cliente['es_academico'].sum()

1832

In [6]:
cliente['es_trabajador'].sum()

6654